## This is a custom function Lat-Lon to Tract (Blocks)

#### Convert the Latitude and Longitude to a Geolocation Tract i.e. Taxi Zone ID. 
#### For performing this, the NYC census data, for blocks etc was used
#### The output is a 15 byte block ID, of which the 6-8 positions form the Taxi Zone.
#### While performing this data extraction it was found that there are new blocks that have been added, apart from the Taxi Zone codes earlier provided

Credits: https://www.kaggle.com/muonneutrino/nyc-taxis-eda-and-mapping-position-to-borough

In [1]:
import pandas as pd
import numpy as np
def get_census_data():
    blocks = pd.read_csv('D:/ISB/Term1/PRAT2/GeoData/census_block_loc.csv')
    census = pd.read_csv('D:/ISB/Term1/PRAT2/GeoData/nyc_census_tracts.csv',index_col=0)
    blocks['Tract'] = blocks.BlockCode // 10000
    blocks = blocks.merge(census,how='left',left_on='Tract',right_index=True)
    return blocks,census

def convert_to_2d(lats,lons,values):
    latmin = 40.48
    lonmin = -74.28
    latmax = 40.93
    lonmax = -73.65
    lon_vals = np.mgrid[lonmin:lonmax:200j]
    lat_vals = np.mgrid[latmin:latmax:200j]
    map_values = np.zeros([200,200])
    dlat = lat_vals[1] - lat_vals[0]
    dlon = lon_vals[1] - lon_vals[0]
    for lat,lon,value in zip(lats,lons,values):
        lat_idx = int(np.rint((lat - latmin) / dlat))
        lon_idx = int(np.rint((lon-lonmin) / dlon ))        
        if not np.isnan(value):
            map_values[lon_idx,lat_idx] = value
    return lat_vals,lon_vals,map_values

blocks,census = get_census_data()
blocks_tmp = blocks[blocks.County_x.isin(['Bronx','Kings','New York','Queens','Richmond'])]
#map_lats, map_lons,map_tracts_nyc = convert_to_2d(blocks_tmp.Latitude,blocks_tmp.Longitude,blocks_tmp.Tract)
map_lats, map_lons,map_tracts = convert_to_2d(blocks.Latitude,blocks.Longitude,blocks.Tract)

def get_tract(lat,lon):
    try:
        latmin = 40.48
        lonmin = -74.28
        latmax = 40.93
        lonmax = -73.65
        dlat = (latmax-latmin) / 199
        dlon = (lonmax-lonmin) / 199
        #print(round(float(lat) * 10000,0))
        #print('here1')
        latI = int(float(lat * 10000))
        latminI = int(float(latmin * 10000))
        latmaxI =int(float(latmax * 10000))
        lonI = int(float(lon * 10000))
        lonminI = int(float(lonmin * 10000))
        lonmaxI = int(float(lonmax * 10000))
        #print('here2')

        #print(latminI)
        #print(latI)
        #print(latmaxI)
        #print(lonminI)
        #print(lonI)
        #print(lonmaxI)
        #if ((int(lat * 10000) - latmin * 10000) >= 0) & ((latmax * 10000 - lat* 10000) >= 0) &  ((lon * 10000 - lonmin * 10000) >= 0) & ((lonmax * 10000 - lon* 10000) >= 0):
        if (latminI <= latI <= latmaxI) and (lonminI <= lonI <= lonmaxI):
            lat_idx = int(np.rint((lat - latmin) / dlat))
            lon_idx = int(np.rint((lon-lonmin) / dlon )) 
            #print('Here {}'.format(map_tracts[lon_idx,lat_idx]))
            return map_tracts[lon_idx,lat_idx]
        return 0
    except:
        pass
        #print('exception')

### Columns were scrambled, the same set of columns werent maintained in all CSVs. Hence a custom CSV extractor was written and also 

In [2]:
import re
import csv
import codecs
import os

for fnamex in os.listdir('D:/ISB/Term1/PRAT2/Green_Cab_Full/'):
    
    f=os.path.join('D:/ISB/Term1/PRAT2/Green_Cab_Full/', fnamex)
    print(f)
    f = open(f, 'r') 
    l = f.readline()
    l = l.lower()
    x = re.split(r'\t', l)
    #print(x)

    vid=99
    ptime=99
    dtime=99
    rcode=99
    plocation=99
    dlocation=99
    pcount=99
    tripd=99
    totamt=99
    ttype=99
    pulong=99
    pulat=99
    dolong=99
    dolat=99

    for l1 in range(0,len(x)):
        if 'vendorid' in x[l1]:
            vid = l1
        if 'lpep_pickup_datetime' in x[l1]:
            ptime = l1
        if 'lpep_dropoff_datetime' in x[l1]:
            dtime = l1
        if 'ratecodeid' in x[l1]:
            rcode = l1
        if 'pulocationid' in x[l1]:
            plocation = l1
        if 'dolocationid' in x[l1]:
            dlocation = l1
        if 'passenger_count' in x[l1]:
            pcount = l1
        if 'trip_distance' in x[l1]:
            tripd = l1
        if 'total_amount' in x[l1]:
            totamt = l1
        if 'trip_type' in x[l1]:
            ttype = l1
        if 'pickup_longitude' in x[l1]:
            pulon=l1
        if 'pickup_latitude' in x[l1]:
            pulat=l1
        if 'dropoff_longitude' in x[l1]:
            dolon=l1
        if 'dropoff_latitude' in x[l1]:
            dolat=l1
            
    f.close()

    fnameox=os.path.join('D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/', fnamex.replace('.csv','-out.csv'))
    fo=open(fnameox, 'w')         
    csvw = csv.writer(fo, delimiter='\t',quotechar='|', quoting=csv.QUOTE_MINIMAL,dialect="excel", lineterminator="\n")

    #f = open('D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2018-06.csv', 'r') 

    f=os.path.join('D:/ISB/Term1/PRAT2/Green_Cab_Full/', fnamex)

    with open(f,'r') as f:
        cr = csv.reader(f)
        for x in cr:
            if(len(x)>0):
                #print(x[0].strip('\t'))
                if(len(x[0].split('\t')) > 2):
                    ax = list(x[0].split('\t'))

                    #print(ax)
                    if vid==99:
                        vidv=0
                    else:
                        vidv=ax[vid]
                    if ptime==99:
                        ptimev=0
                    else:
                        ptimev=ax[ptime]
                    if dtime==99:
                        dtimev=0
                    else:
                        dtimev=ax[dtime]
                    if rcode==99:
                        rcodev=0
                    else:
                        rcodev=ax[rcode]
                    if plocation==99:
                        plocationv=0
                    else:
                        plocationv=ax[plocation]
                    if dlocation==99:
                        dlocationv=0
                    else:
                        dlocationv=ax[dlocation]
                    if pcount==99:
                        pcountv=0
                    else:
                        pcountv=ax[pcount]
                    if tripd==99:
                        tripdv=0
                    else:
                        tripdv=ax[tripd]
                    if totamt==99:
                        totamtv=0
                    else:
                        totamtv=ax[totamt]
                    if ttype==99:
                        ttypev=0
                    else:
                        ttypev=ax[ttype]
                    if pulon != 99 & plocation==99:
                        #print('Lat:{}, Lon:{}'.format(ax[pulat],ax[pulon]))
                        try:
                            #print('x:{},y:{}'.format(ax[pulat],ax[pulon]))
                            pulatI = int(float(ax[pulat]))
                            pulonI = int(float(ax[pulon]))
                            #print('a:{},b:{}'.format(pulatI,pulonI))
                            if int(pulatI) !=0:
                                #print('Lat:{}, Lon:{}'.format(ax[pulat],ax[pulon]))
                                plocationv = str(get_tract(float(ax[pulat]),float(ax[pulon])))[5:8]
                        except:
                            pass
                        #print(plocationv)
                    if dolon != 99 & dlocation==99:
                        try:
                            dolatI = int(float(ax[dolat]))
                            dolonI = int(float(ax[dolon]))
                            if dolatI !=0:
                                dlocationv = str(get_tract(float(ax[dolat]),float(ax[dolon])))[5:8]
                        except:
                            pass
                        #print(dlocationv)
                                     
                    outr=str(vidv)+'\t'+str(ptimev)+'\t'+str(dtimev)+'\t'+str(rcodev)+'\t'+str(plocationv)+'\t'+str(dlocationv)+'\t'+str(pcountv)+'\t'+str(tripdv)+'\t'+str(totamtv)+'\t'+str(ttypev)
                    csvw.writerow([outr])
    fo.close()

D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2013-08.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2013-09.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2013-10.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2013-11.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2013-12.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-01.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-02.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-03.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-04.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-05.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-06.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-07.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-08.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-09.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-10.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-11.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2014-12.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2015-01.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full/green_2015-02.csv
D:/ISB/Term1

#### Replacing junk characters

In [3]:
### import re
import csv
import codecs
import os
import fileinput


for fnamex in os.listdir('D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/'):
    
    f=os.path.join('D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/', fnamex)
    fo=os.path.join('D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean2/', fnamex)
    print(f)
    #f = open(f, 'r') 
    with fileinput.FileInput(f, inplace=False) as file, open(fo, 'w') as outfile:
        for line in file:
            line = line.replace('|', '')
            outfile.write(line)
        outfile.close()

D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2013-08-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2013-09-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2013-10-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2013-11-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2013-12-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-01-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-02-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-03-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-04-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-05-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-06-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-07-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-08-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-09-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-10-out.csv
D:/ISB/Term1/PRAT2/Green_Cab_Full_Clean/green_2014-11-out.csv
D:/ISB/T